In [466]:
import matplotlib
import matplotlib.pyplot as plt

import os

import torch
import torch.nn as nn
import torch.optim as optim

import torchvision
from torchvision import datasets, models, transforms

import numpy as np
import time
from tqdm import tqdm

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") # device 객체

In [467]:
import pandas as pd

test_dir = '/opt/ml/input/cropped_v2/eval'
submission = pd.read_csv(os.path.join(test_dir, 'info.csv'))

In [468]:
# 데이터셋을 불러올 때 사용할 변형(transformation) 객체 정의
transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    #transforms.Resize((112, 112)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]) # 정규화(normalization)
])

## 이 아래부터 data_dir을 하나씩 바꾸어 가면서 반복 (단 첫줄에 있는 기존의 data_dir은 사용하지 않는다.)

In [541]:
# 이 부분 중 각각의 3가지를 반복해서 돌리면 된다.
# data_dir = '/opt/ml/input/cropped_v2/train/images_classified/' ; ft = 'ans';
data_dir = '/opt/ml/input/cropped_v2/train/images_classified_mask/' ; ft = 'mask';
# data_dir = '/opt/ml/input/cropped_v2/train/images_classified_gender/' ; ft = 'gender';
# data_dir = '/opt/ml/input/cropped_v2/train/images_classified_age/' ; ft = 'age';
dataset = datasets.ImageFolder(os.path.join(data_dir), transforms)

In [542]:
dataset

Dataset ImageFolder
    Number of datapoints: 18837
    Root location: /opt/ml/input/cropped_v2/train/images_classified_mask/
    StandardTransform
Transform: Compose(
               Resize(size=(224, 224), interpolation=PIL.Image.BILINEAR)
               ToTensor()
               Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
           )

In [543]:
class_names = dataset.classes
print(class_names)

['0', '12', '6']


In [544]:
train_size = int(0.8 * len(dataset))
valid_size = len(dataset) - train_size

In [545]:
train_set, valid_set = torch.utils.data.random_split(dataset, [train_size, valid_size], generator=torch.Generator().manual_seed(42))

In [546]:
#dataloader = torch.utils.data.DataLoader(dataset, batch_size=4, shuffle=True, num_workers=8)
dataloader = torch.utils.data.DataLoader(train_set, batch_size=128, shuffle=True, num_workers=8)

In [547]:
def imshow(input, title):
    # torch.Tensor를 numpy 객체로 변환
    # pytorch는 batch, channel, 높이, 너비 순으로 데이터 구성.
    input = input.numpy().transpose((1, 2, 0)) # channel 값이 가장 뒤에 오도록.
    # 이미지 정규화 해제하기
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    input = std * input + mean
    input = np.clip(input, 0, 1)
    # 이미지 출력
    plt.imshow(input)
    plt.title(title)
    plt.show()

'''
# 학습 데이터를 배치 단위로 불러오기
iterator = iter(dataloader)

# 현재 배치를 이용해 격자 형태의 이미지를 만들어 시각화
inputs, classes= next(iterator)
out = torchvision.utils.make_grid(inputs)
imshow(out, title=[class_names[x] for x in classes])
'''

'\n# 학습 데이터를 배치 단위로 불러오기\niterator = iter(dataloader)\n\n# 현재 배치를 이용해 격자 형태의 이미지를 만들어 시각화\ninputs, classes= next(iterator)\nout = torchvision.utils.make_grid(inputs)\nimshow(out, title=[class_names[x] for x in classes])\n'

In [548]:
# !pip install ipywidgets
# !jupyter nbextension enable --py widgetsnbextension

In [549]:
model = models.resnet34(pretrained=True)

num_features = model.fc.in_features
# 전이 학습(transfer learning): 모델의 출력 뉴런 수를 18개로 교체하여 마지막 레이어 다시 학습
model.fc = nn.Linear(num_features, len(class_names)) 
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
# optimizer = optim.Adam(model.parameters(), lr=0.001)

In [550]:
# import timm
# class Network(nn.Module):
#     def __init__(self, num_classes):
#         super().__init__()
#         self.encoder = timm.create_model('regnety_040', pretrained=True,
#                                     drop_path_rate=0.20,
#                                     )
        
#         num_head = self.encoder.head.fc.in_features
#         self.encoder.head.fc = nn.Linear(num_head, num_classes)

#     def forward(self, x):
#         x = self.encoder(x)
#         return x
# model = Network(len(class_names))
# model = model.to(device)
# criterion = nn.CrossEntropyLoss()
# # optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
# optimizer = optim.Adam(model.parameters(), lr=0.001)

In [551]:
#num_epochs = 50
num_epochs = 10
model.train()
start_time = time.time()

# 전체 반복(epoch) 수 만큼 반복하며
for epoch in range(num_epochs):
    running_loss = 0.
    running_corrects = 0

    # 배치 단위로 학습 데이터 불러오기
    for inputs, labels in tqdm(dataloader):
        inputs = inputs.to(device)
        labels = labels.to(device)

        # 모델에 입력(forward)하고 결과 계산
        optimizer.zero_grad() # 전체 grad 값을 초기화.
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        loss = criterion(outputs, labels)

        # 역전파를 통해 기울기(gradient) 계산 및 학습 진행
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * inputs.size(0)
        running_corrects += torch.sum(preds == labels.data)

    epoch_loss = running_loss / len(train_set)
    epoch_acc = running_corrects / len(train_set) * 100.

    # 학습 과정 중에 결과 출력
    print('#{} Loss: {:.4f} Acc: {:.4f}% Time: {:.4f}s'.format(epoch, epoch_loss, epoch_acc, time.time() - start_time))

  0%|          | 0/118 [00:00<?, ?it/s]

#0 Loss: 0.0460 Acc: 98.3078% Time: 28.1854s


  0%|          | 0/118 [00:00<?, ?it/s]

#1 Loss: 0.0143 Acc: 99.6284% Time: 56.3502s


  0%|          | 0/118 [00:00<?, ?it/s]

...

#7 Loss: 0.0158 Acc: 99.5686% Time: 225.2498s


  0%|          | 0/118 [00:00<?, ?it/s]

#8 Loss: 0.0089 Acc: 99.7346% Time: 253.6600s


100%|██████████| 118/118 [00:27<00:00,  4.24it/s]

#9 Loss: 0.0044 Acc: 99.8673% Time: 281.5299s


In [552]:
valid_dataloader = torch.utils.data.DataLoader(valid_set, batch_size=128, shuffle=True, num_workers=8)

In [553]:
model.eval()
start_time = time.time()

all_labels = []
all_preds = []

with torch.no_grad():
    running_loss = 0.
    running_corrects = 0

    for inputs, labels in valid_dataloader:
        inputs = inputs.to(device)
        labels = labels.to(device)

        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        #all_preds.append(preds)
        
        for p in preds.cpu().numpy():
            all_preds.append(p)
        
        loss = criterion(outputs, labels)

        running_loss += loss.item() * inputs.size(0)
        running_corrects += torch.sum(preds == labels.data)
        
        
        #all_labels.append(labels.data)
        
        for l in labels.cpu().data.numpy():
            all_labels.append(l)
        
        # 한 배치의 첫 번째 이미지에 대하여 결과 시각화
#         print(f'[예측 결과: {class_names[preds[0]]}] (실제 정답: {class_names[labels.data[0]]})')
#         imshow(inputs.cpu().data[0], title='예측 결과: ' + class_names[preds[0]])

    epoch_loss = running_loss / len(valid_set)
    epoch_acc = running_corrects / len(valid_set) * 100.
    print('[Test Phase] Loss: {:.4f} Acc: {:.4f}% Time: {:.4f}s'.format(epoch_loss, epoch_acc, time.time() - start_time))

[Test Phase] Loss: 0.0285 Acc: 99.1242% Time: 3.4933s


In [554]:
#all_preds

In [555]:
#all_labels

In [556]:
# !conda install -c conda-forge torchmetrics -y

In [557]:
from sklearn.metrics import f1_score
#from torchmetrics import F1Score

#f1 = F1Score(num_classes=18)
#f1(all_preds, all_labels)
f1_score(all_labels, all_preds, average='macro')

0.987797452906556

In [558]:
# import pandas as pd

# test_dir = '/opt/ml/input/cropped_v2/eval'
# submission = pd.read_csv(os.path.join(test_dir, 'info.csv'))
#맨위로 올려보냄

In [559]:
image_dir = os.path.join(test_dir, 'images')

In [560]:
submission

,ImageID,ans,age,gender
0,cbc5c6e168e63498590db46022617123f1fe1268.jpg,0,1,0
1,0e72482bf56b3581c081f7da2a6180b8792c7089.jpg,0,1,0
2,b549040c49190cedc41327748aeb197c1670f14d.jpg,0,1,0
3,4f9cb2a045c6d5b9e50ad3459ea7b791eb6e18bc.jpg,0,1,0
4,248428d9a4a5b6229a7081c32851b90cb8d38d0c.jpg,0,0,0
...,...,...,...,...
12595,d71d4570505d6af8f777690e63edfa8d85ea4476.jpg,0,1,0
12596,6cf1300e8e218716728d5820c0bab553306c2cfd.jpg,0,1,3
12597,8140edbba31c3a824e817e6d5fb95343199e2387.jpg,0,0,3
12598,030d439efe6fb5a7bafda45a393fc19f2bf57f54.jpg,0,1,0


In [561]:
image_paths = [os.path.join(image_dir, img_id) for img_id in submission.ImageID]

In [562]:
#image_paths

In [563]:
from torch.utils.data import Dataset, DataLoader

class TestDataset(Dataset):
    def __init__(self, img_paths, transform):
        self.img_paths = img_paths
        self.transform = transform

    def __getitem__(self, index):
        image = Image.open(self.img_paths[index])

        if self.transform:
            image = self.transform(image)
        return image

    def __len__(self):
        return len(self.img_paths)

In [564]:
test_set = TestDataset(image_paths, transforms)

In [565]:
test_dataloader = torch.utils.data.DataLoader(test_set, batch_size=128, shuffle=False, num_workers=8)

In [566]:
def imshow_test(input):
    # torch.Tensor를 numpy 객체로 변환
    # pytorch는 batch, channel, 높이, 너비 순으로 데이터 구성.
    input = input.numpy().transpose((1, 2, 0)) # channel 값이 가장 뒤에 오도록.
    # 이미지 정규화 해제하기
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    input = std * input + mean
    input = np.clip(input, 0, 1)
    # 이미지 출력
    plt.imshow(input)
    #plt.title(title)
    plt.show()

In [567]:
'''
# 데이터를 배치 단위로 불러오기
iterator = iter(test_dataloader)

# 현재 배치를 이용해 격자 형태의 이미지를 만들어 시각화
inputs = next(iterator)
out = torchvision.utils.make_grid(inputs)
imshow_test(out)
'''

'\n# 데이터를 배치 단위로 불러오기\niterator = iter(test_dataloader)\n\n# 현재 배치를 이용해 격자 형태의 이미지를 만들어 시각화\ninputs = next(iterator)\nout = torchvision.utils.make_grid(inputs)\nimshow_test(out)\n'

In [568]:
from PIL import Image
all_predictions = []

with torch.no_grad():
    for inputs in tqdm(test_dataloader):
        inputs = inputs.to(device)
        
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        
        all_predictions.extend(preds.cpu().numpy())

100%|██████████| 99/99 [00:09<00:00, 10.31it/s]


In [569]:
'''
from PIL import Image

all_predictions = []
for images in tqdm(test_dataloader):
    with torch.no_grad():
        images = images.to(device)
        pred = model(images)
        pred = pred.argmax(dim=-1)
        all_predictions.extend(pred.cpu().numpy())
'''

'\nfrom PIL import Image\n\nall_predictions = []\nfor images in tqdm(test_dataloader):\n    with torch.no_grad():\n        images = images.to(device)\n        pred = model(images)\n        pred = pred.argmax(dim=-1)\n        all_predictions.extend(pred.cpu().numpy())\n'

In [570]:
for i in range(len(all_predictions)):
    print(all_predictions[i], end=' ')
    
    if i % 5 == 4:
        print()
    
    if i == 19:
        break

1 0 1 1 1 
0 2 0 0 0 
1 0 2 0 0 
0 0 0 2 0 


In [571]:
class_names

['0', '12', '6']

In [572]:
all_predictions2 = []

for p in all_predictions:
    all_predictions2.append(class_names[p])

In [573]:
for i in range(len(all_predictions2)):
    print(all_predictions2[i], end=' ')
    
    if i % 5 == 4:
        print()
    
    if i == 19:
        break

12 0 12 12 12 
0 6 0 0 0 
12 0 6 0 0 
0 0 0 6 0 


In [574]:
submission[ft] = all_predictions2

In [575]:
# torch.save(model.state_dict(), os.getcwd()+'/'+f'2nd_model_run_{ft}')

In [576]:
submission['ans'].value_counts()

0    12600
Name: ans, dtype: int64

## 따로따로 계산시 여기 위까지 반복

In [577]:
submission

,ImageID,ans,age,gender,mask
0,cbc5c6e168e63498590db46022617123f1fe1268.jpg,0,1,0,12
1,0e72482bf56b3581c081f7da2a6180b8792c7089.jpg,0,1,0,0
2,b549040c49190cedc41327748aeb197c1670f14d.jpg,0,1,0,12
3,4f9cb2a045c6d5b9e50ad3459ea7b791eb6e18bc.jpg,0,1,0,12
4,248428d9a4a5b6229a7081c32851b90cb8d38d0c.jpg,0,0,0,12
...,...,...,...,...,...
12595,d71d4570505d6af8f777690e63edfa8d85ea4476.jpg,0,1,0,0
12596,6cf1300e8e218716728d5820c0bab553306c2cfd.jpg,0,1,3,0
12597,8140edbba31c3a824e817e6d5fb95343199e2387.jpg,0,0,3,6
12598,030d439efe6fb5a7bafda45a393fc19f2bf57f54.jpg,0,1,0,0


In [578]:
submission = submission.astype({'age':'int','gender':'int','mask':'int'})
submission.dtypes

ImageID    object
ans         int64
age         int64
gender      int64
mask        int64
dtype: object

In [579]:
try : submission['ans'] = submission['mask'] + submission['gender'] + submission['age'] ; submission2 = submission.drop(['mask','gender','age'],axis=1)
except : pass
submission2

,ImageID,ans
0,cbc5c6e168e63498590db46022617123f1fe1268.jpg,13
1,0e72482bf56b3581c081f7da2a6180b8792c7089.jpg,1
2,b549040c49190cedc41327748aeb197c1670f14d.jpg,13
3,4f9cb2a045c6d5b9e50ad3459ea7b791eb6e18bc.jpg,13
4,248428d9a4a5b6229a7081c32851b90cb8d38d0c.jpg,12
...,...,...
12595,d71d4570505d6af8f777690e63edfa8d85ea4476.jpg,1
12596,6cf1300e8e218716728d5820c0bab553306c2cfd.jpg,4
12597,8140edbba31c3a824e817e6d5fb95343199e2387.jpg,9
12598,030d439efe6fb5a7bafda45a393fc19f2bf57f54.jpg,1


In [580]:
# a = []
# for i in range(len(submission)):
#     b = int(submission['ans'][i][-1]) + int(submission['ans'][i][-2]) + int(submission['ans'][i][:-2])
#     a.append(b)
# submission['ans'] = a

In [581]:

submission2.to_csv(os.path.join(test_dir, 'submission_res_sepera_adam.csv'), index=False)

In [465]:
#model.state_dict()

In [ ]:
os.getcwd()

In [221]:
# torch.save(model.state_dict(), os.getcwd()+'/'+'2nd_model_run')